
    
# Converting a TensorFlow* Model in Linux*

## Pre-Requisites:


    
### Install Tensorflow 1.12 version

Run the following cells to install tensorflow 1.12 version

In [ ]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.

In [ ]:
pip install --user tensorflow==1.12

Note: you may need to restart the kernel to use updated packages.

Create new temporary directory by running the following cell

In [ ]:
! export TMPDIR=/tmp/$USER; mkdir -p $TMPDIR; python -c "import tempfile; print(tempfile.gettempdir())"

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name squeezenet1.1 -o models

### Download Models from TensorFlow* -slim library

There are a number of pre-trained public models in the TensorFlow*-slim repository. The models are distributed as Python scripts and checkpoint files. First of all download repository with models. Note: This is done in home directory(~)

In [ ]:
! cd && git clone https://github.com/tensorflow/models/ && cd models/research/slim


    
### Export Inference Graph and download checkpoint file

Export inference graph for one of the available models using the following command (Inception V1 in this example):

In [ ]:
! python3 -W ignore  $HOME/models/research/slim/export_inference_graph.py --alsologtostderr --model_name=inception_v1 --output_file=/tmp/$USER/inception_v1_inf_graph.pb  


    
The script creates inference graph file with name “inception_v1_inf_graph.pb” in the /tmp direcory.

### Download archive with checkpoint file (Inception V1 in this example):

In [ ]:
%%writefile run.sh
export CHECKPOINT_DIR=/tmp/$USER/checkpoints

mkdir $CHECKPOINT_DIR

wget http://download.tensorflow.org/models/inception_v1_2016_08_28.tar.gz

tar -xvf inception_v1_2016_08_28.tar.gz

mv inception_v1.ckpt $CHECKPOINT_DIR

rm inception_v1_2016_08_28.tar.gz

In [ ]:
%%bash
sh run.sh


    
### Freeze the model

The last step is to freeze the graph. To do this you need to know the output node of the model you are planning to freeze. This information is found by running the summarize_graph.

### Summarize Graph

Go to ~/models/research/slim/ directory and run summarize_graph.py script.

In [ ]:
! cd $HOME/models/research/slim/ && python3 -W ignore  /opt/intel/openvino/deployment_tools/model_optimizer/mo/utils/summarize_graph.py --input_model=/tmp/$USER/inception_v1_inf_graph.pb 



    
The output layer/node name should be in the last line of text on the console and should look like:

1 input(s) detected: Name: input, type: float32, shape: (-1,224,224,3) 1 output(s) detected: InceptionV1/Logits/Predictions/Reshape_1

### Freeze the graph for Intel® Distribution of OpenVINO™ toolkit

The script generates inception_v1_frozen.pb file with the frozen model in the directory you are currently in.

In [ ]:
! python3 -W ignore /usr/local/lib/python3.5/dist-packages/tensorflow/python/tools/freeze_graph.py --input_graph /tmp/$USER/inception_v1_inf_graph.pb --input_binary --input_checkpoint /tmp/$USER/checkpoints/inception_v1.ckpt --output_node_names InceptionV1/Logits/Predictions/Reshape_1 --output_graph inception_v1_frozen.pb


    
You might get warning message "Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA" while executing above command, ignore that.
## Convert Frozen Tensorflow* model to IR using Model Optimizer

Assuming you are in the ~/models/research/slim/ directory

In [ ]:
! python3 -W ignore  /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --input_model inception_v1_frozen.pb --input_shape [1,224,224,3] --mean_values [1024,1024,1024] --scale_values [128,128,128]


    
This should produce “inception_v1_frozen.xml” and “inception_v1_frozen.bin” file. The xml file contains the topology information of the model and the bin file contains the model’s weights and biases. These two files are expected when using the inference engine and so make note of the path.

## Run Classification Sample

The classification sample will showcase the Intel® Distribution of OpenVINO™ toolkit inference engine using TensorFlow model Inception_v1_frozen IR files (.xml & .bin) and an input image of a car to classify. The classification collateral is defined as the input image car_1.bmp, the Inception_v1_frozen IR files (.xml & .bin), and the labels file inception_v1_frozen.labels. Create a new directory that will hold the classification sample app and all needed components to run the classification sample Note: The following steps should be followed and are assuming you are following the preceding steps. You should be in the home directory.

In [ ]:
!/opt/intel/openvino/bin/setupvars.sh

In [ ]:
! cp /opt/intel/openvino/deployment_tools/demo/squeezenet1.1.labels ./inception_v1_frozen.labels >/dev/null 2>&1


    
#### Run Application

Note: To see all the flags that the sample takes as input, run the below cell

In [ ]:
! python3 classification_sample.py -h

We will use the [car_1.png](car_1.png) image below to capture attributes 

In [ ]:
! python3 classification_sample.py -i car_1.png --labels inception_v1_frozen.labels -m inception_v1_frozen.xml



Expected Output:


 
<img src="classification_output.png"> 
